In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset
news = load_dataset("multi_news", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
news.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


In [4]:
train_news = news.train_test_split(test_size=0.2)

In [5]:
!pip install transformers accelerate

In [6]:
from transformers import AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained('t5-small')

In [8]:
def preprocessing(examples):
  input = ['summary'+doc for doc in examples['document']]
  model_input = tokenizer(input, max_length=1024, truncation=True)
  label = tokenizer(examples["summary"], max_length=1024, truncation=True)
  model_input['label'] = label['input_ids']
  return model_input

In [9]:
tokenized = train_news.map(preprocessing, batched=True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [10]:
tokenized

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'label'],
        num_rows: 4497
    })
    test: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'label'],
        num_rows: 1125
    })
})

In [11]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5-small')

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [14]:
training_arg = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate = 2e-2,
    per_device_train_batch_size = 10,
    per_device_eval_batch_size = 10,
    weight_decay = .01,
    num_train_epochs = 10,
    fp16 = True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_arg,
    train_dataset = tokenized['train'],
    eval_dataset = tokenized['test'],
    tokenizer = tokenizer,
    data_collator = data_collator
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 538.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 77.06 MiB is free. Process 10340 has 14.67 GiB memory in use. Of the allocated memory 13.76 GiB is allocated by PyTorch, and 791.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)